<a href="https://colab.research.google.com/github/Athugodage/RuLawSimplification/blob/main/trained_models/ru_t5_test(rouge_metrics)_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Athugodage/RuLawSimplification.git

Cloning into 'RuLawSimplification'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 168 (delta 41), reused 29 (delta 8), pack-reused 76
Receiving objects: 100% (168/168), 43.82 MiB | 17.31 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [ ]:
import json

with open('/content/RuLawSimplification/two_way/two_way.json', 'r', encoding='utf8') as rp:
    a = json.load(rp)

In [ ]:
a[100]

In [ ]:
import pandas as pd

df = pd.DataFrame({'Text': [], 'Comment': []})


for article in a:
    for sujet in article['article text']:
        df = df.append({'Text': sujet['text'], 'Comment': sujet['comment']}, ignore_index=True)


In [ ]:
df

In [ ]:
df.to_csv('two_way_trial.csv', index=False)

In [ ]:
df.shape

In [ ]:
df.iloc[:2500].to_csv('train.csv', index=False, header=None)
df.iloc[2500:2800].to_csv('validation.csv', index=False, header=None)
df.iloc[2800:].to_csv('test.csv', index=False, header=None)

In [ ]:
!pip install Sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 35.3 MB/s 
     |████████████████████████████████| 163 kB 49.0 MB/s 


In [ ]:
!pip install huggingface_hub


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 5.2 MB/s 
     |████████████████████████████████| 115 kB 36.4 MB/s 
     |████████████████████████████████| 212 kB 51.8 MB/s 
     |████████████████████████████████| 127 kB 48.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
pd.read_csv('test.csv', encoding='utf8', sep=',', index_col=False, header=None)

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from datasets import load_dataset


dataset = load_dataset('csv', 
                       data_files={'train': ['train.csv'],
                                   'validation': ['validation.csv'],
                                   'test': ['test.csv']},
                       encoding='utf8',
                       index_col = False,
                       header=None,
                       sep=','
                      )

FileNotFoundError: ignored

In [ ]:
print(dataset['train'])
print(dataset['validation'])
print(dataset['test'])

In [ ]:
dataset.shape

In [ ]:
dataset.push_to_hub('marcus2000/twowaydata')

In [ ]:
from datasets import load_dataset

dataset = load_dataset('marcus2000/twowaydata')

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Downloading:   0%|          | 0.00/892 [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/marcus2000___parquet/marcus2000--twowaydata-70c3f9955e1f2e89/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset['train']

Dataset({
    features: ['0', '1'],
    num_rows: 2500
})

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"], 2)

,0,1
0,"4) абзацы второй - четырнадцатый пункта 22 статьи 1 Федерального закона от 10 января 2003 года N 8-ФЗ ""О внесении изменений и дополнений в Закон Российской Федерации ""О занятости населения в Российской Федерации"" и отдельные законодательные акты Российской Федерации по вопросам финансирования мероприятий по содействию занятости населения"" (Собрание законодательства Российской Федерации, 2003, N 2, ст. 160);","То есть для того, чтобы получить предложение о работе, приходить на прием к трудовому инспектору не обязательно, приглашение безработному направят в личный кабинет на портале ""Работа в России""."
1,2) на плановый период 2023 и 2024 годов согласно приложению 4 к настоящему Федеральному закону.,Они будут представлены в ближайшее время.


In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoConfig


tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/rut5_base_sum_gazeta")



Downloading:   0%|          | 0.00/279 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/828k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
max_input_length = 100
max_target_length = 100


def preprocess_function(examples):
    inputs = [ex for ex in examples['0']]  # 0 - Текст
    targets = [ex for ex in examples['1']]  # 1 - Комментарий
    model_inputs = tokenizer(inputs, 
                             max_length=max_input_length,
                             truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    test: Dataset({
        features: ['0', '1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 492
    })
    train: Dataset({
        features: ['0', '1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2500
    })
    validation: Dataset({
        features: ['0', '1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
})

In [ ]:
for x in tokenized_datasets['train']:
    print(x['0'], x['input_ids'])

Указанное понимание не соотносится и с буквальным смыслом абзаца второго пункта 1 статьи 134 Закона о банкротстве, закрепляющего единственный критерий внеочередного удовлетворения требований кредитора - направленность платежей на предотвращение катастроф либо гибели людей, вне зависимости от того, связано ли обязательство по внесению платежей с исполнением обязанностей по договору, заключенному в рамках обычной хозяйственной деятельности должника, или нет. [867, 14796, 1863, 425, 6413, 401, 917, 7325, 2170, 259, 279, 388, 259, 13311, 9912, 259, 11267, 637, 259, 29074, 16527, 4587, 893, 6293, 308, 333, 5530, 279, 8402, 5654, 308, 259, 411, 3321, 12389, 10109, 261, 374, 10678, 1933, 18671, 259, 10526, 1292, 17217, 543, 10992, 13773, 27086, 17444, 1454, 13790, 1347, 4311, 6903, 259, 264, 13245, 2763, 12961, 1006, 310, 1910, 18037, 10042, 16264, 259, 5607, 2272, 14687, 2668, 1006, 261, 10992, 259, 9535, 279, 552, 259, 2107, 261, 4011, 7861, 2357, 13572, 2023, 425, 315, 13685, 748, 12961, 1

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("IlyaGusev/rut5_base_sum_gazeta")

Downloading:   0%|          | 0.00/766 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/977M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

batch_size = 16

args = Seq2SeqTrainingArguments("ru_t5model_for_legalsimplification",
    evaluation_strategy = "epoch",
    learning_rate=2e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    push_to_hub=True,
    predict_with_generate=True,
    fp16=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=ecdb35e26882bd66dcdae1492f81182da9dcbf06db0fb106ee167c6557c14fe7
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [ ]:
from datasets import load_metric


metric = load_metric("rouge")

In [ ]:
import numpy as np
import nltk
nltk.download('punkt')


import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/repository.py:708: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.11.
  FutureWarning,
Cloning https://huggingface.co/marcus2000/ru_t5model_for_legalsimplification into local empty directory.
Using cuda_amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2500
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1570
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad`

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,nan,0.536400,0.148100,0.506000,0.491700,163.030000
2,No log,nan,0.536400,0.148100,0.506000,0.491700,163.030000
3,No log,nan,0.536400,0.148100,0.506000,0.491700,163.030000
4,0.000000,nan,0.536400,0.148100,0.506000,0.491700,163.030000
5,0.000000,nan,0.536400,0.148100,0.506000,0.491700,163.030000
6,0.000000,nan,0.536400,0.148100,0.506000,0.491700,163.030000
7,0.000000,nan,0.536400,0.148100,0.506000,0.491700,163.030000
8,0.000000,nan,0.536400,0.148100,0.506000,0.491700,163.030000
9,0.000000,nan,0.536400,0.148100,0.506000,0.491700,163.030000
10,0.000000,nan,0.536400,0.148100,0.506000,0.491700,163.030000


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and hav

TrainOutput(global_step=1570, training_loss=0.0, metrics={'train_runtime': 2196.7547, 'train_samples_per_second': 11.38, 'train_steps_per_second': 0.715, 'total_flos': 3316403840901120.0, 'train_loss': 0.0, 'epoch': 10.0})

In [ ]:
predidcted = trainer.predict(tokenized_datasets['test'])

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 492
  Batch size = 16


In [ ]:
predidcted

PredictionOutput(predictions=array([[    2,  2285, 16586, ...,     0,     0,     0],
       [    2,  1635,  8497, ...,  6186,  1006,  1121],
       [    2,  2285, 16586, ...,     0,     0,     0],
       ...,
       [    2,  1145, 15978, ...,  6268,   259,   279],
       [    2,  1145, 15978, ...,  6268,   259,   279],
       [    2, 11513, 29611, ...,     0,     0,     0]]), label_ids=array([[ 2117, 12298,  1451, ...,     0,     0,     0],
       [  259,  3060,  1923, ...,     0,     0,     0],
       [ 1414, 12626,  2872, ...,     0,     0,     0],
       ...,
       [  259, 19575, 13688, ...,     0,     0,     0],
       [  688,  4892,   259, ...,     0,     0,     0],
       [ 2095,  1923,  1648, ...,     0,     0,     0]]), metrics={'test_loss': nan, 'test_rouge1': 0.7683, 'test_rouge2': 0.0, 'test_rougeL': 0.7806, 'test_rougeLsum': 0.7477, 'test_gen_len': 156.5833, 'test_runtime': 262.0821, 'test_samples_per_second': 1.877, 'test_steps_per_second': 0.118})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to ru_t5model_for_legalsimplification
Configuration saved in ru_t5model_for_legalsimplification/config.json
Model weights saved in ru_t5model_for_legalsimplification/pytorch_model.bin
tokenizer config file saved in ru_t5model_for_legalsimplification/tokenizer_config.json
Special tokens file saved in ru_t5model_for_legalsimplification/special_tokens_map.json
Copy vocab file to ru_t5model_for_legalsimplification/spiece.model
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/Oct01_01-02-48_aeb38811de6d/events.out.tfevents.1664586350.aeb38811de6d.64.0:  34%|###3      …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/marcus2000/ru_t5model_for_legalsimplification
   7f8c40d..c084672  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/marcus2000/ru_t5model_for_legalsimplification
   7f8c40d..c084672  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 0.5364}]}
To https://huggingface.co/marcus2000/ru_t5model_for_legalsimplification
   c084672..d2551bf  main -> main

   c084672..d2551bf  main -> main



'https://huggingface.co/marcus2000/ru_t5model_for_legalsimplification/commit/c0846723380ae9e2664d19d5b0a166ff908c59bd'

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.7 MB/s 


In [ ]:
import evaluate
# evaluate.list_evaluation_modules(module_type='metric', with_details=True)

In [ ]:
metric_value = 8.1  # value obtained from https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/stt_en_conformer_ctc_small 

evaluate.push_to_hub(
    model_id='marcus2000/fine_tuned_t5_model',
    task_type="automatic-speech-recognition",
    dataset_type="librispeech_asr",
    dataset_name="Librispeech (clean)",
    metric_type="wer",
    metric_name="WER",
    dataset_split="test",  # corresponds to test-clean set
    dataset_config="other",  # corresponds to test-clean set
    dataset_args=dict(language="en"),  # metadata for dataset
    # the actual score obtained by the model
    metric_value=metric_value,
)

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

KeyError: ignored